In [1]:
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics
import sys
sys.path.append(os.path.abspath('..'))

from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy
from Generate_Samples import generate_samples

In [2]:
## Load dataset
dataset_orig = pd.read_csv('../data/GermanData.csv')

## Drop categorical features
dataset_orig = dataset_orig.drop(['1','2','4','5','8','10','11','12','14','15','16','17','18','19','20'],axis=1)

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A91', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A92', 0, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A93', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A94', 1, dataset_orig['sex'])
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'A95', 0, dataset_orig['sex'])

# mean = dataset_orig.loc[:,"age"].mean()
# dataset_orig['age'] = np.where(dataset_orig['age'] >= mean, 1, 0)
dataset_orig['age'] = np.where(dataset_orig['age'] >= 25, 1, 0)
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A30', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A31', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A32', 1, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A33', 2, dataset_orig['credit_history'])
dataset_orig['credit_history'] = np.where(dataset_orig['credit_history'] == 'A34', 3, dataset_orig['credit_history'])

dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A61', 1, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A62', 1, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A63', 2, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A64', 2, dataset_orig['savings'])
dataset_orig['savings'] = np.where(dataset_orig['savings'] == 'A65', 3, dataset_orig['savings'])

dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A72', 1, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A73', 1, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A74', 2, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A75', 2, dataset_orig['employment'])
dataset_orig['employment'] = np.where(dataset_orig['employment'] == 'A71', 3, dataset_orig['employment'])



## ADD Columns
dataset_orig['credit_history=Delay'] = 0
dataset_orig['credit_history=None/Paid'] = 0
dataset_orig['credit_history=Other'] = 0

dataset_orig['credit_history=Delay'] = np.where(dataset_orig['credit_history'] == 1, 1, dataset_orig['credit_history=Delay'])
dataset_orig['credit_history=None/Paid'] = np.where(dataset_orig['credit_history'] == 2, 1, dataset_orig['credit_history=None/Paid'])
dataset_orig['credit_history=Other'] = np.where(dataset_orig['credit_history'] == 3, 1, dataset_orig['credit_history=Other'])

dataset_orig['savings=500+'] = 0
dataset_orig['savings=<500'] = 0
dataset_orig['savings=Unknown/None'] = 0

dataset_orig['savings=500+'] = np.where(dataset_orig['savings'] == 1, 1, dataset_orig['savings=500+'])
dataset_orig['savings=<500'] = np.where(dataset_orig['savings'] == 2, 1, dataset_orig['savings=<500'])
dataset_orig['savings=Unknown/None'] = np.where(dataset_orig['savings'] == 3, 1, dataset_orig['savings=Unknown/None'])

dataset_orig['employment=1-4 years'] = 0
dataset_orig['employment=4+ years'] = 0
dataset_orig['employment=Unemployed'] = 0

dataset_orig['employment=1-4 years'] = np.where(dataset_orig['employment'] == 1, 1, dataset_orig['employment=1-4 years'])
dataset_orig['employment=4+ years'] = np.where(dataset_orig['employment'] == 2, 1, dataset_orig['employment=4+ years'])
dataset_orig['employment=Unemployed'] = np.where(dataset_orig['employment'] == 3, 1, dataset_orig['employment=Unemployed'])


dataset_orig = dataset_orig.drop(['credit_history','savings','employment'],axis=1)
## In dataset 1 means good, 2 means bad for probability. I change 2 to 0
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 2, 0, 1)

protected_attribute = 'sex'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)

default_score_dict = {"accuracy":[],
                       "F1":[],
                      "aod age":[],
                      "eod age":[],
                      "SPD age":[],
                      "DI  age":[],
                     "aod sex":[],
                      "eod sex":[],
                      "SPD sex":[],
                      "DI  sex":[]}

improved_score_dict = {"accuracy":[],
                       "F1":[],
                      "aod age":[],
                      "eod age":[],
                      "SPD age":[],
                      "DI  age":[],
                     "aod sex":[],
                      "eod sex":[],
                      "SPD sex":[],
                      "DI  sex":[]}

# Fair-SMOTE

In [3]:
for i in range(5):
    
    dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, shuffle = True)

    X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
    X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

    clf = LogisticRegression()

    default_score_dict["accuracy"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'accuracy'))
    default_score_dict["F1"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'F1'))
    default_score_dict["aod age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'aod'))
    default_score_dict["eod age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'eod'))
    default_score_dict["SPD age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'SPD'))
    default_score_dict["DI  age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'DI'))

    default_score_dict["aod sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'aod'))
    default_score_dict["eod sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'eod'))
    default_score_dict["SPD sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'SPD'))
    default_score_dict["DI  sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'DI'))
    
    # Find Class & Protected attribute Distribution
    # first one is class value and second one is protected attribute value
    zero_zero = len(dataset_orig_train[(dataset_orig_train['Probability'] == 0) & (dataset_orig_train[protected_attribute] == 0)])
    zero_one = len(dataset_orig_train[(dataset_orig_train['Probability'] == 0) & (dataset_orig_train[protected_attribute] == 1)])
    one_zero = len(dataset_orig_train[(dataset_orig_train['Probability'] == 1) & (dataset_orig_train[protected_attribute] == 0)])
    one_one = len(dataset_orig_train[(dataset_orig_train['Probability'] == 1) & (dataset_orig_train[protected_attribute] == 1)])

    print(zero_zero,zero_one,one_zero,one_one)
    
    # Sort these four
    
    maximum = max(zero_zero,zero_one,one_zero,one_one)
    if maximum == zero_zero:
        print("zero_zero is maximum")
    if maximum == zero_one:
        print("zero_one is maximum")
    if maximum == one_zero:
        print("one_zero is maximum")
    if maximum == one_one:
        print("one_one is maximum")
        
    zero_zero_to_be_incresed = maximum - zero_zero ## where both are 0
    zero_one_to_be_incresed = maximum - zero_one ## where class is 0 attribute is 1
    one_zero_to_be_incresed = maximum - one_zero ## where class is 1 attribute is 0


    print(zero_zero_to_be_incresed,zero_one_to_be_incresed,one_zero_to_be_incresed)

    df_zero_zero = dataset_orig_train[(dataset_orig_train['Probability'] == 0) & (dataset_orig_train[protected_attribute] == 0)]
    df_zero_one = dataset_orig_train[(dataset_orig_train['Probability'] == 0) & (dataset_orig_train[protected_attribute] == 1)]
    df_one_zero = dataset_orig_train[(dataset_orig_train['Probability'] == 1) & (dataset_orig_train[protected_attribute] == 0)]



    df_zero_zero = generate_samples(zero_zero_to_be_incresed,df_zero_zero,'German')
    df_zero_one = generate_samples(zero_one_to_be_incresed,df_zero_one,'German')
    df_one_zero = generate_samples(one_zero_to_be_incresed,df_one_zero,'German')
    
    
    df = df_zero_zero.append(df_zero_one)
    df = df.append(df_one_zero)

    df['sex'] = df['sex'].astype(float)

    df_one_one = dataset_orig_train[(dataset_orig_train['Probability'] == 1) & (dataset_orig_train[protected_attribute] == 1)]
    df = df.append(df_one_one)
    
    # Check score after Fair-SMOTE
    
    X_train, y_train = df.loc[:, df.columns != 'Probability'], df['Probability']    

    clf = LogisticRegression() # LSR
    
    improved_score_dict["accuracy"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'accuracy'))
    improved_score_dict["F1"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'F1'))
    improved_score_dict["aod age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'aod'))
    improved_score_dict["eod age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'eod'))
    improved_score_dict["SPD age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'SPD'))
    improved_score_dict["DI  age"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "age", 'DI'))

    improved_score_dict["aod sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'aod'))
    improved_score_dict["eod sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'eod'))
    improved_score_dict["SPD sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'SPD'))
    improved_score_dict["DI  sex"].append(measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, "sex", 'DI'))
    


86 146 165 403
one_one is maximum
317 257 238
88 149 162 401
one_one is maximum
313 252 239
81 153 168 398
one_one is maximum
317 245 230
95 158 150 397
one_one is maximum
302 239 247
89 151 163 397
one_one is maximum
308 246 234


# Analyze score

In [4]:
import statistics

print("Default Scores")
print("---------------")

print("accuracy ",statistics.median(default_score_dict['accuracy']))
print("F1 ",statistics.median(default_score_dict['F1']))
print("aod sex ",statistics.median(default_score_dict['aod sex']))
print("eod sex ",statistics.median(default_score_dict['eod sex']))
print("SPD sex ",statistics.median(default_score_dict['SPD sex']))
print("DI  sex",statistics.median(default_score_dict['DI  sex']))
print("aod age ",statistics.median(default_score_dict['aod age']))
print("eod age ",statistics.median(default_score_dict['eod age']))
print("SPD age ",statistics.median(default_score_dict['SPD age']))
print("DI  age",statistics.median(default_score_dict['DI  age']))


print("---------------------------------------------------------------------")


print("Fair-SMOTE Scores")
print("---------------")


print("accuracy ",statistics.median(improved_score_dict['accuracy']))
print("F1 ",statistics.median(improved_score_dict['F1']))
print("aod sex ",statistics.median(improved_score_dict['aod sex']))
print("eod sex ",statistics.median(improved_score_dict['eod sex']))
print("SPD sex ",statistics.median(improved_score_dict['SPD sex']))
print("DI  sex",statistics.median(improved_score_dict['DI  sex']))
print("aod age ",statistics.median(improved_score_dict['aod age']))
print("eod age ",statistics.median(improved_score_dict['eod age']))
print("SPD age ",statistics.median(improved_score_dict['SPD age']))
print("DI  age",statistics.median(improved_score_dict['DI  age']))


Default Scores
---------------
accuracy  0.68
F1  0.81
aod sex  -0.04
eod sex  0.06
SPD sex  0.08
DI  sex 0.08
aod age  -0.05
eod age  0.1
SPD age  0.16
DI  age 0.16
---------------------------------------------------------------------
Fair-SMOTE Scores
---------------
accuracy  0.62
F1  0.68
aod sex  -0.02
eod sex  0.01
SPD sex  0.07
DI  sex 0.06
aod age  0.01
eod age  0.25
SPD age  0.23
DI  age 0.45
